In [ ]:
import json
import pandas as pd
import numpy as np

# Build csv files for all xps

sender_type = ['sender_rdma', 'sender_rdma_write_multip_read']
payload_sizes = ['10-100', '100-1000']
simult_req = ['1']
payload_bounds = ['1500']
circle_sizes = ['200']
n_runs = 5

data = []

for size in payload_sizes:
   for n in simult_req:
    for payload_bound in payload_bounds:
            for circle_size in circle_sizes:
        # one xp, let's get th latencies and avg req rates
        #   CASE   OPS/SEC ...
        # sender | res1 res2 res3        
                #For custom senders
                for sender in sender_type:
                    ops_sec = []
                    avg_latency = []
                    p50_00_latency = []
                    p99_00_latency = []
                    p99_99_latency = []

                    for i in range(n_runs):
                        result_json = f"results2/json_{sender}_{size}_{n}_{payload_bound}_{circle_size}_{i}"
                        # print(result_json)    
                        with open(result_json, 'r') as file:
                            result =  json.load(file)                
                            
                            # result_dict = {}
                            # result_dict["sender_type"] = sender[7:]
                            # result_dict["payload_size"] = size
                            # result_dict["simult_req"] = n
                            # result_dict["payload_bound"] = payload_bound
                            # result_dict["circle_size"] = circle_size
                            ops_sec.append(result['ALL STATS']['Totals']['Ops/sec'])
                            avg_latency.append(result['ALL STATS']['Totals']['Average Latency'])
                            p50_00_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p50.00'])
                            p99_00_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p99.00'])
                            p99_99_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p99.90'])
                            
                            # data.append(result_dict)
                            # global_data.append(result_dict)
                    # print(result)  
                    result_dict = {}
                    result_dict["sender_type"] = sender[7:]
                    result_dict["payload_size"] = size
                    result_dict["simult_req"] = n
                    result_dict["payload_bound"] = payload_bound
                    result_dict["circle_size"] = circle_size

                    result_dict["Ops/sec"] = np.mean(ops_sec)
                    result_dict["Average"] = np.mean(avg_latency)
                    result_dict["p50.00"] = np.mean(p50_00_latency)
                    result_dict["p99.00"] = np.mean(p99_00_latency)
                    result_dict["p99.90"] = np.mean(p99_99_latency)

                    result_dict["std_Ops/sec"] = np.std(ops_sec)
                    result_dict["std_Average"] = np.std(avg_latency)
                    result_dict["std_p50.00"] = np.std(p50_00_latency)
                    result_dict["std_p99.00"] = np.std(p99_00_latency)
                    result_dict["std_p99.90"] = np.std(p99_99_latency)

                    data.append(result_dict)
                # #For tcp_proxy    
                # result_json = f"results/json_tcpproxy_{size}_{n}"
                # # print(result_json)            
                # with open(result_json, 'r') as file:
                #     result =  json.load(file)                
                    
                #     result_dict = {}
                #     result_dict["sender_type"] = "tcpproxy"
                #     result_dict["payload_size"] = size
                #     result_dict["simult_req"] = n
                #     result_dict["payload_bound"] = payload_bound
                #     result_dict["circle_size"] = circle_size
                #     result_dict["Ops/sec"] = result['ALL STATS']['Totals']['Ops/sec']
                #     result_dict["Average"] = result['ALL STATS']['Totals']['Average Latency']
                #     result_dict["p50.00"] = result['ALL STATS']['Totals']['Percentile Latencies']['p50.00']
                #     result_dict["p99.00"] = result['ALL STATS']['Totals']['Percentile Latencies']['p99.00']
                #     result_dict["p99.90"] = result['ALL STATS']['Totals']['Percentile Latencies']['p99.90']
                    
                #     data.append(result_dict)
                #     global_data.append(result_dict)
                # print(result)  

                # print("\n\n\n")
                # df = pd.DataFrame(data)
                # df.to_csv(f'data2/res_{size}_{n}_{payload_bound}_{circle_size}.csv')

df = pd.DataFrame(data)
df.to_csv(f'data2/res_global.csv')

In [ ]:
# Build (sender_type, payload_size) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data2/res_global.csv')

simult_req = [1]
payload_bounds = [1500]
circle_sizes = [200]
metrics = ['p99.90']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for n in simult_req:
    for payload_bound in payload_bounds:
        for circle_size in circle_sizes:
            for metric in metrics:
                plt.clf() # Reset plot
                df_fixed = df[(df['simult_req'] == n) & (df['payload_bound'] == payload_bound) & (df['circle_size'] == circle_size)]

                df_comp_fixed_temp = df_fixed.pivot("sender_type", "payload_size", metric)
                yerr = df_fixed.pivot("sender_type", "payload_size", f"std_{metric}")
                ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

                # Set plot title and labels
                plt.title(f"{metric} Latency comparison \n for simult_req={n}, payload_bound={payload_bound}, circle_size={circle_size}")
                plt.xlabel('Protoype')
                plt.ylabel(f"{metric} Latency (ms)")

                # Orient sender_type labels at 45 degrees
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                ax.tick_params(axis='x', pad=10)

                # Adjust plot layout
                plt.legend(title='Payload Size', bbox_to_anchor=(1, 1))
                plt.tight_layout()

                # Show or save the plot
                # plt.savefig(f'plots/sender-comp-payload_size-{n}_{payload_bound}_{circle_size}_{metric}.svg', bbox_inches='tight', format='svg')
                plt.show()

In [ ]:
import json
import pandas as pd
import numpy as np

# Build csv files for all xps

sender_type = ['sender_rdma', 'sender_rdma_write_multip_write', 'sender_rdma_write_multip_read', 'sender_rdma_write_nomultip_read', 'tcpproxy']
payload_sizes = ['10-100','100-1000','1000-10000']
simult_req = ['1','10','100']
payload_bounds = ['1500']
circle_sizes = ['200']
n_runs = 5
data = []

for size in payload_sizes:
   for n in simult_req:
    for payload_bound in payload_bounds:
        for circle_size in circle_sizes:
            for sender in sender_type:
                ops_sec = []
                avg_latency = []
                p50_00_latency = []
                p99_00_latency = []
                p99_99_latency = []

                for i in range(n_runs):
                    if sender != "tcpproxy":
                        result_json = f"results/json_{sender}_{size}_{n}_{payload_bound}_{circle_size}_{i}"
                    else:
                        result_json = f"results/json_tcpproxy_{size}_{n}"
                    # print(result_json)    
                    with open(result_json, 'r') as file:
                        result =  json.load(file)                
                        
                        ops_sec.append(result['ALL STATS']['Totals']['Ops/sec'])
                        avg_latency.append(result['ALL STATS']['Totals']['Average Latency'])
                        p50_00_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p50.00'])
                        p99_00_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p99.00'])
                        p99_99_latency.append(result['ALL STATS']['Totals']['Percentile Latencies']['p99.90'])
                        
                result_dict = {}
                result_dict["sender_type"] = sender
                result_dict["payload_size"] = size
                result_dict["simult_req"] = n
                result_dict["payload_bound"] = payload_bound
                result_dict["circle_size"] = circle_size

                result_dict["Ops/sec"] = np.mean(ops_sec)
                result_dict["Average"] = np.mean(avg_latency)
                result_dict["p50.00"] = np.mean(p50_00_latency)
                result_dict["p99.00"] = np.mean(p99_00_latency)
                result_dict["p99.90"] = np.mean(p99_99_latency)

                result_dict["std_Ops/sec"] = np.std(ops_sec)
                result_dict["std_Average"] = np.std(avg_latency)
                result_dict["std_p50.00"] = np.std(p50_00_latency)
                result_dict["std_p99.00"] = np.std(p99_00_latency)
                result_dict["std_p99.90"] = np.std(p99_99_latency)

                data.append(result_dict)

df = pd.DataFrame(data)
df.to_csv(f'data/res_global.csv')

In [ ]:
# Build (sender_type, payload_size) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_global.csv')

simult_req = [1, 10, 100]
payload_bounds = [1500]
circle_sizes = [200]
metrics = ['p99.90', 'p99.00', 'p50.00', 'Average']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for n in simult_req:
    for payload_bound in payload_bounds:
        for circle_size in circle_sizes:
            for metric in metrics:
                plt.clf() # Reset plot
                df_fixed = df[(df['simult_req'] == n) & (df['payload_bound'] == payload_bound) & (df['circle_size'] == circle_size)]

                df_comp_fixed_temp = df_fixed.pivot("sender_type", "payload_size", metric)
                yerr = df_fixed.pivot("sender_type", "payload_size", f"std_{metric}")
                ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

                # Set plot title and labels
                plt.title(f"{metric} Latency comparison \n for simult_req={n}, payload_bound={payload_bound}, circle_size={circle_size}")
                plt.xlabel('Protoype')
                plt.ylabel(f"{metric} Latency (ms)")

                # Orient sender_type labels at 45 degrees
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                ax.tick_params(axis='x', pad=10)

                # Adjust plot layout
                plt.legend(title='Payload Size', bbox_to_anchor=(1, 1))
                plt.tight_layout()

                # Save the plot
                plt.savefig(f'plots/sender-comp-payload_size-{n}_{payload_bound}_{circle_size}_{metric}.svg', bbox_inches='tight', format='svg')

In [ ]:
# Build (sender_type, simult_req) comparison plots

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/res_global.csv')

payload_sizes = ['10-100','100-1000','1000-10000']
payload_bounds = [1500]
circle_sizes = [200]
metrics = ['p99.90', 'p99.00', 'p50.00', 'Average']

# Set plot style
plt.style.use('seaborn-whitegrid')

# Compare average throughput
for size in payload_sizes:
    for payload_bound in payload_bounds:
        for circle_size in circle_sizes:
            for metric in metrics:
                plt.clf() # Reset plot
                df_fixed = df[(df['payload_size'] == size) & (df['payload_bound'] == payload_bound) & (df['circle_size'] == circle_size)]

                df_comp_fixed_temp = df_fixed.pivot("sender_type", "simult_req", metric)
                yerr = df_fixed.pivot("sender_type", "simult_req", f"std_{metric}")
                ax = df_comp_fixed_temp.plot(kind='bar', rot=0, yerr=yerr, capsize=4)

                # Set plot title and labels
                plt.title(f"{metric} Latency comparison \n for payload_size={size}, payload_bound={payload_bound}, circle_size={circle_size}")
                plt.xlabel('Protoype')
                plt.ylabel(f"{metric} Latency (ms)")

                # Orient sender_type labels at 45 degrees
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                ax.tick_params(axis='x', pad=10)

                # Adjust plot layout
                plt.legend(title='Simultaneous Requests', bbox_to_anchor=(1, 1))
                plt.tight_layout()

                # Save the plot
                plt.savefig(f'plots/sender-comp-simult_req-{size}_{payload_bound}_{circle_size}_{metric}.svg', bbox_inches='tight', format='svg')